In [ ]:
# Import Library
import os
from os import listdir
from PIL import Image as Img
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
import zipfile
from keras.models import load_model
import numpy as np
import tensorflow as tf
import pickle
import cv2

In [ ]:
# Connect To Google Drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


#Face Detec

In [ ]:
# Install YoloV8
!pip install ultralytics
from ultralytics import YOLO

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.5/593.5 kB 11.0 MB/s eta 0:00:00


In [ ]:
# Check Yolo Version
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.113 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


In [ ]:
# Extract Dataset
!unzip '/content/drive/MyDrive/UTS_DL_T3/Dataset/dataset.zip' -d '/content/drive/MyDrive/UTS_DL_T3/Dataset/'

Archive:  /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset.zip
 extracting: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/classes.txt  
  inflating: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/data.yaml  
   creating: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/train/
   creating: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/train/images/
  inflating: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/train/images/Face (1).jpg  
  inflating: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/train/images/Face (10).jpg  
  inflating: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/train/images/Face (2).jpg  
  inflating: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/train/images/Face (3).jpg  
  inflating: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/train/images/Face (4).jpg  
  inflating: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/train/images/Face (5).jpg  
  inflating: /content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/train/images/Face (6).jpg  
  inflating

In [ ]:
# Change Directory
%cd /content/drive/MyDrive/UTS_DL_T3/Weight

/content/drive/MyDrive/UTS_DL_T3/Weight


In [ ]:
# Training the Model for Face Detection
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/data.yaml epochs=50 imgsz=640 plots=True

100% 21.5M/21.5M [00:00<00:00, 134MB/s] 
Ultralytics YOLOv8.0.113 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=T

#Face Recog

In [ ]:
# Import HaarCascade
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
# Import FaceNet
!pip install keras-facenet
from keras_facenet import FaceNet
MyFaceNet = FaceNet()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.4 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10370 sha256=7a00dd2d522b1430f296c0362bc845a8fafe8f645304a7dde1b80b6531aa75d2
  Stored in directory: /root/.cache/pip/wheels/1d/d8/a9/85cf04ea29321d2afcb82c0caaafdca9195385f9d68cbc7185
Successfully built keras-facenet


In [ ]:
# Set Directory
folder = '/content/drive/MyDrive/UTS_DL_T3/Dataset/dataset/valid/images/'

In [ ]:
# Training the Model for Face Recog
database = {}

for filename in listdir(folder):

    path = folder + filename
    gbr1 = cv2.imread(folder + filename)

    wajah = HaarCascade.detectMultiScale(gbr1,1.1,4)

    if len(wajah)>0:
        x1, y1, width, height = wajah[0]
    else:
        x1, y1, width, height = 1, 1, 10, 10

    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Img.fromarray(gbr)
    gbr_array = asarray(gbr)

    face = gbr_array[y1:y2, x1:x2]

    face = Img.fromarray(face)
    face = face.resize((160,160))
    face = asarray(face)

    face = expand_dims(face, axis=0)
    signature = MyFaceNet.embeddings(face)

    database[os.path.splitext(filename)[0]]=signature

1/1 [==============================] - 0s 28ms/step


In [ ]:
# Change Directory
%cd /content/drive/MyDrive/UTS_DL_T3/Weight/

/content/drive/MyDrive/UTS_DL_T3/Weight


In [ ]:
# Save File Model for Face Recog
myfile = open("data.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()